In [1]:
import os
import av
import soundfile as sf
import numpy as np
from tqdm import tqdm

def ensure_fft_size(audio_data, sample_rate, max_fft_size=512):
    """
    确保FFT size不超过指定值（通过降采样实现）
    返回调整后的音频数据和采样率
    """
    if sample_rate > max_fft_size:
        # 计算合适的降采样比率
        downsample_ratio = int(sample_rate / max_fft_size)
        new_sample_rate = sample_rate // downsample_ratio
        audio_data = audio_data[::downsample_ratio]
        print(f"降采样: {sample_rate}Hz -> {new_sample_rate}Hz 以满足FFT size要求")
        return audio_data, new_sample_rate
    return audio_data, sample_rate

def m4a_to_wav(input_file, output_file, max_fft_size=512):
    """转换单个文件并确保FFT size符合要求"""
    try:
        # 读取M4A文件
        container = av.open(input_file)
        audio_stream = next(s for s in container.streams if s.type == 'audio')
        
        # 提取音频数据
        frames = []
        for frame in container.decode(audio_stream):
            frames.append(frame.to_ndarray())
        
        audio_data = np.concatenate(frames)
        original_rate = audio_stream.rate
        
        # 确保FFT size符合要求
        audio_data, new_sample_rate = ensure_fft_size(audio_data, original_rate, max_fft_size)
        
        # 保存为WAV文件
        sf.write(output_file, audio_data, new_sample_rate, subtype='PCM_16')
        
        # 验证输出文件
        if os.path.exists(output_file):
            info = sf.info(output_file)
            actual_fft_size = info.samplerate
            print(f"转换成功: {os.path.basename(input_file)} -> {os.path.basename(output_file)}")
            print(f"采样率: {original_rate}Hz -> {new_sample_rate}Hz")
            print(f"实际FFT size: {actual_fft_size}")
            return True
        return False
    except Exception as e:
        print(f"转换失败 {os.path.basename(input_file)}: {str(e)}")
        return False

def batch_convert_and_clean(input_folder, output_folder=None, max_fft_size=512):
    """
    批量转换并清理原始文件
    """
    # 设置输出路径
    if output_folder is None:
        output_folder = input_folder
    else:
        os.makedirs(output_folder, exist_ok=True)
    
    # 获取M4A文件列表
    m4a_files = [f for f in os.listdir(input_folder) if f.lower().endswith('.m4a')]
    
    if not m4a_files:
        print("未找到.m4a文件")
        return
    
    print(f"找到 {len(m4a_files)} 个.m4a文件，开始转换...")
    
    # 处理文件
    success_count = 0
    for filename in tqdm(m4a_files, desc="处理进度"):
        input_path = os.path.join(input_folder, filename)
        output_filename = os.path.splitext(filename)[0] + '.wav'
        output_path = os.path.join(output_folder, output_filename)
        
        # 转换文件
        if m4a_to_wav(input_path, output_path, max_fft_size):
            # 删除原始文件
            try:
                os.remove(input_path)
                print(f"已删除原始文件: {filename}")
                success_count += 1
            except Exception as e:
                print(f"删除失败 {filename}: {str(e)}")
    
    print(f"\n转换完成! 成功处理 {success_count}/{len(m4a_files)} 个文件")

if __name__ == "__main__":
    # 用户交互
    input_dir = input("请输入包含.m4a文件的文件夹路径: ").strip()
    while not os.path.isdir(input_dir):
        print("路径无效!")
        input_dir = input("请重新输入: ").strip()
    
    custom_output = input("使用不同输出文件夹? (y/n): ").lower().strip()
    output_dir = input("输出文件夹路径: ").strip() if custom_output == 'y' else None
    
    # 确认删除操作
    confirm = input("转换后将删除原始.m4a文件，确认吗? (y/n): ").lower().strip()
    if confirm != 'y':
        print("操作已取消")
        exit()
    
    # 执行转换
    batch_convert_and_clean(input_dir, output_dir)
    
    # 显示最终结果
    if output_dir:
        print(f"\n转换后的WAV文件保存在: {os.path.abspath(output_dir)}")
    else:
        print(f"\n转换后的WAV文件保存在原文件夹: {os.path.abspath(input_dir)}")

找到 9 个.m4a文件，开始转换...


处理进度: 100%|██████████| 9/9 [00:00<00:00, 109.63it/s]

降采样: 48000Hz -> 516Hz 以满足FFT size要求
转换成功: behindyou.m4a -> behindyou.wav
采样率: 48000Hz -> 516Hz
实际FFT size: 516
删除失败 behindyou.m4a: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\Museum\\OneDrive\\作业存档\\人工智能\\audio_data\\behindyou.m4a'
降采样: 48000Hz -> 516Hz 以满足FFT size要求
转换成功: goodbye-2.m4a -> goodbye-2.wav
采样率: 48000Hz -> 516Hz
实际FFT size: 516
删除失败 goodbye-2.m4a: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\Museum\\OneDrive\\作业存档\\人工智能\\audio_data\\goodbye-2.m4a'
降采样: 48000Hz -> 516Hz 以满足FFT size要求
转换成功: goodbye.m4a -> goodbye.wav
采样率: 48000Hz -> 516Hz
实际FFT size: 516
删除失败 goodbye.m4a: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\Museum\\OneDrive\\作业存档\\人工智能\\audio_data\\goodbye.m4a'
降采样: 48000Hz -> 516Hz 以满足FFT size要求
转换成功: hello-2.m4a -> hello-2.wav
采样率: 48000Hz -> 516Hz
实际FFT size: 516
删除失败 hello-2.m4a: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\Museum\\OneDrive\\作业存档\\人工智能\\audio_data\\hello-2.m4a'
降采样: 48000Hz -> 516Hz 以满足FFT size要求
转换成功: hello.m4a -> hello.wav
采样率: 48